# Build Chains and Results  
Instructions: https://fsworks.atlassian.net/wiki/spaces/~558841686/pages/edit-v2/2087550977

In [2]:
import subprocess
import time
data_file_name = 'EMS_DCMA_DD_23.08.graphml'
experiment = None
tasks_types = 'milestones'
results = 'prt'
query = 'code' 
if not experiment: experiment = data_file_name.replace('.graphml', '').replace('.', '_')
counts = []
for i in range(10):
    from calculate import *
    print('run', i+1)
    chains_count, rows_count = run_calculation_process(data_file_name, experiment, tasks_types, results, query)
    counts.append((i+1, chains_count, rows_count))
    time.sleep(60)
counts_df = pd.DataFrame(counts, columns=['run', 'chains', 'rows_count'])
display(counts_df)

Starting EC2 instance: i-0586e11281d4b02a2
EC2 instance "i-0586e11281d4b02a2" has been started
run 1
Data file uploaded to S3
Calculation finished
Compute instance stopped
Preparing results
The results have been downloaded to EMS_DCMA_DD_23_08
9710 chains written to chains.parquet
111782 tasks rows written to parquet files in the prt sub-directory
run 2
Data file uploaded to S3


SSHException: SSH session not active

# Query Results  

In [ ]:
import duckdb
con = duckdb.connect()
prt_files_path = os.path.join(experiment, 'prt')
results_files = "'{e}/*.parquet'".format(e=prt_files_path)
query = """SELECT * FROM {rf} LIMIT 10""".format(rf=results_files)
query_df = con.execute(query).df()
# To save the query results run:
#query_df.to_excel('query_df.xlsx')
display(query_df)

# Read and Write Chains  

In [ ]:
chains_path = os.path.join(experiment, 'chains.parquet')
chains_df = pd.read_parquet(chains_path)
chains = list(chains_df['Chain'])
chains = '\n'.join(chains)+'\n'
with open('chains.txt', 'w') as f: f.write(chains)
display(chains_df.head())
display(chains_df.info())